# Colab inference for ZFTurbo's [Music-Source-Separation-Training](https://github.com/ZFTurbo/Music-Source-Separation-Training/)


<font size=1>*made by [jarredou](https://github.com/jarredou)*</font>


In [ ]:
import base64
#@markdown # Install

%cd /content
!git clone https://github.com/ZFTurbo/Music-Source-Separation-Training

#generate new requirements.txt file for faster colab install
req = 'IyB0b3JjaCAjPT0yLjAuMQ0KbnVtcHkNCnBhbmRhcw0Kc2NpcHkNCnNvdW5kZmlsZQ0KbWxfY29sbGVjdGlvbnMNCnRxZG0NCnNlZ21lbnRhdGlvbl9tb2RlbHNfcHl0b3JjaD09MC4zLjMNCnRpbW09PTAuOS4yDQphdWRpb21lbnRhdGlvbnM9PTAuMjQuMA0KcGVkYWxib2FyZD09MC44LjENCm9tZWdhY29uZj09Mi4yLjMNCmJlYXJ0eXBlPT0wLjE0LjENCnJvdGFyeV9lbWJlZGRpbmdfdG9yY2g9PTAuMy41DQplaW5vcHM9PTAuNi4xDQpsaWJyb3NhDQpkZW11Y3MgIz09NC4wLjANCiMgdHJhbnNmb3JtZXJzPT00LjM1LjANCnRvcmNobWV0cmljcz09MC4xMS40DQpzcGFmZT09MC4zLjINCnByb3RvYnVmPT0zLjIwLjMNCnRvcmNoX2F1ZGlvbWVudGF0aW9ucw0KYXN0ZXJvaWQ9PTAuNy4wDQphdXJhbG9zcw0KdG9yY2hzZWcNCg=='
dec_req = base64.b64decode(req).decode('utf-8')
f = open("Music-Source-Separation-Training/requirements.txt", "w")
f.write(dec_req)
f.close()

!mkdir '/content/Music-Source-Separation-Training/ckpts'

print('Installing the dependencies... This will take few minutes')
!pip install -r 'Music-Source-Separation-Training/requirements.txt' &> /dev/null

print('Installation is done !')

In [ ]:
#@markdown #Gdrive connection
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/Music-Source-Separation-Training/'
import os
import torch
import yaml

class IndentDumper(yaml.Dumper):
    def increase_indent(self, flow=False, indentless=False):
        return super(IndentDumper, self).increase_indent(flow, False)


def tuple_constructor(loader, node):
    # Load the sequence of values from the YAML node
    values = loader.construct_sequence(node)
    # Return a tuple constructed from the sequence
    return tuple(values)

# Register the constructor with PyYAML
yaml.SafeLoader.add_constructor('tag:yaml.org,2002:python/tuple',
tuple_constructor)



def conf_edit(config_path, chunk_size, overlap):
    print("Using custom overlap/chunk_size values")
    with open(config_path, 'r') as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)


    data['audio']['chunk_size'] = chunk_size
    data['inference']['num_overlap'] = overlap

    with open(config_path, 'w') as f:
        yaml.dump(data, f, default_flow_style=False, sort_keys=False, Dumper=IndentDumper, allow_unicode=True)



def download_file(url):
    path= 'ckpts'
    os.makedirs(path, exist_ok=True)
    filename = os.path.basename(url)
    file_path = os.path.join(path, filename)

    if os.path.exists(file_path):
        print(f"File '{filename}' already exists at '{path}'.")
        return

    try:
        response = torch.hub.download_url_to_file(url, file_path)
        print(f"File '{filename}' downloaded successfully")
    except Exception as e:
        print(f"Error downloading file '{filename}' from '{url}': {e}")


#@markdown # Separation
#@markdown #### Separation config:
input_folder = '/content/drive/MyDrive/input' #@param {type:"string"}
output_folder = '/content/drive/MyDrive/output' #@param {type:"string"}
model = 'VOCALS-MelBand-Roformer (by KimberleyJSN)' #@param ['VOCALS-MelBand-Roformer (by KimberleyJSN)', 'VOCALS-InstVocHQ', 'VOCALS-BS-Roformer_1297 (by viperx)', 'VOCALS-BS-Roformer_1296 (by viperx)', 'KARAOKE-MelBand-Roformer (by aufr33 & viperx)', 'OTHER-BS-Roformer_1053 (by viperx)','4STEMS-SCNet_MUSDB18 (by starrytong)', 'CROWD-REMOVAL-MelBand-Roformer (by aufr33)', 'VOCALS-VitLarge23 (by ZFTurbo)', 'CINEMATIC-BandIt_Plus (by kwatcharasupat)', 'DRUMSEP-MDX23C_DrumSep_6stem (by aufr33 & jarredou)', 'DE-REVERB-MDX23C (by aufr33 & jarredou)', 'DENOISE-MelBand-Roformer-1 (by aufr33)', 'DENOISE-MelBand-Roformer-2 (by aufr33)']
extract_instrumental = True #@param {type:"boolean"}
export_format = 'wav FLOAT' #@param ['wav FLOAT', 'flac PCM_16', 'flac PCM_24']
use_tta = False #@param {type:"boolean"}
#@markdown ---
#@markdown *Roformers custom config:*
overlap = 2 #@param {type:"slider", min:1, max:40, step:1}
chunk_size = 352800 #@param [352800, 485100] {type:"raw"}

if export_format.startswith('flac'):
    flac_file = True
    pcm_type = export_format.split(' ')[1]
else:
    flac_file = False
    pcm_type = None

if model == 'VOCALS-InstVocHQ':
    model_type = 'mdx23c'
    config_path = 'ckpts/config_vocals_mdx23c.yaml'
    start_check_point = 'ckpts/model_vocals_mdx23c_sdr_10.17.ckpt'
    download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/config_vocals_mdx23c.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.0/model_vocals_mdx23c_sdr_10.17.ckpt')


elif model == 'VOCALS-MelBand-Roformer (by KimberleyJSN)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/config_vocals_mel_band_roformer_kj.yaml'
    start_check_point = 'ckpts/MelBandRoformer.ckpt'
    download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/KimberleyJensen/config_vocals_mel_band_roformer_kj.yaml')
    download_file('https://huggingface.co/KimberleyJSN/melbandroformer/resolve/main/MelBandRoformer.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-BS-Roformer_1297 (by viperx)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/model_bs_roformer_ep_317_sdr_12.9755.yaml'
    start_check_point = 'ckpts/model_bs_roformer_ep_317_sdr_12.9755.ckpt'
    download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/viperx/model_bs_roformer_ep_317_sdr_12.9755.yaml')
    download_file('https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_317_sdr_12.9755.ckpt')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-BS-Roformer_1296 (by viperx)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/model_bs_roformer_ep_368_sdr_12.9628.yaml'
    start_check_point = 'ckpts/model_bs_roformer_ep_368_sdr_12.9628.ckpt'
    download_file('https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_368_sdr_12.9628.ckpt')
    download_file('https://raw.githubusercontent.com/TRvlvr/application_data/main/mdx_model_data/mdx_c_configs/model_bs_roformer_ep_368_sdr_12.9628.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'KARAOKE-MelBand-Roformer (by aufr33 & viperx)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/config_mel_band_roformer_karaoke.yaml'
    start_check_point = 'ckpts/mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33-viperx-karaoke-bsroformer-model/resolve/main/mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33-viperx-karaoke-bsroformer-model/resolve/main/config_mel_band_roformer_karaoke.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'OTHER-BS-Roformer_1053 (by viperx)':
    model_type = 'bs_roformer'
    config_path = 'ckpts/model_bs_roformer_ep_937_sdr_10.5309.yaml'
    start_check_point = 'ckpts/model_bs_roformer_ep_937_sdr_10.5309.ckpt'
    download_file('https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_937_sdr_10.5309.ckpt')
    download_file('https://raw.githubusercontent.com/TRvlvr/application_data/main/mdx_model_data/mdx_c_configs/model_bs_roformer_ep_937_sdr_10.5309.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'CROWD-REMOVAL-MelBand-Roformer (by aufr33)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/model_mel_band_roformer_crowd.yaml'
    start_check_point = 'ckpts/mel_band_roformer_crowd_aufr33_viperx_sdr_8.7144.ckpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.4/mel_band_roformer_crowd_aufr33_viperx_sdr_8.7144.ckpt')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.4/model_mel_band_roformer_crowd.yaml')
    conf_edit(config_path, chunk_size, overlap)

elif model == 'VOCALS-VitLarge23 (by ZFTurbo)':
    model_type = 'segm_models'
    config_path = 'ckpts/config_vocals_segm_models.yaml'
    start_check_point = 'ckpts/model_vocals_segm_models_sdr_9.77.ckpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.0/config_vocals_segm_models.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v1.0.0/model_vocals_segm_models_sdr_9.77.ckpt')

elif model == 'CINEMATIC-BandIt_Plus (by kwatcharasupat)':
    model_type = 'bandit'
    config_path = 'ckpts/config_dnr_bandit_bsrnn_multi_mus64.yaml'
    start_check_point = 'ckpts/model_CINEMATIC-BandIt_Plus (by kwatcharasupat)_dnr_sdr_11.47.chpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.3/config_dnr_bandit_bsrnn_multi_mus64.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.3/model_CINEMATIC-BandIt_Plus (by kwatcharasupat)_dnr_sdr_11.47.chpt')

elif model == 'DRUMSEP-MDX23C_DrumSep_6stem (by aufr33 & jarredou)':
    model_type = 'mdx23c'
    config_path = 'ckpts/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.yaml'
    start_check_point = 'ckpts/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.ckpt'
    download_file('https://github.com/jarredou/models/releases/download/aufr33-jarredou_MDX23C_DrumSep_model_v0.1/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.ckpt')
    download_file('https://github.com/jarredou/models/releases/download/aufr33-jarredou_MDX23C_DrumSep_model_v0.1/aufr33-jarredou_DrumSep_model_mdx23c_ep_141_sdr_10.8059.yaml')

elif model == '4STEMS-SCNet_MUSDB18 (by starrytong)':
    model_type = 'scnet'
    config_path = 'ckpts/config_musdb18_scnet.yaml'
    start_check_point = 'ckpts/scnet_checkpoint_musdb18.ckpt'
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.6/config_musdb18_scnet.yaml')
    download_file('https://github.com/ZFTurbo/Music-Source-Separation-Training/releases/download/v.1.0.6/scnet_checkpoint_musdb18.ckpt')

elif model == 'DE-REVERB-MDX23C (by aufr33 & jarredou)':
    model_type = 'mdx23c'
    config_path = 'ckpts/config_dereverb_mdx23c.yaml'
    start_check_point = 'ckpts/ dereverb_mdx23c_sdr_6.9096.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33_jarredou_MDXv3_DeReverb/resolve/main/dereverb_mdx23c_sdr_6.9096.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33_jarredou_MDXv3_DeReverb/resolve/main/config_dereverb_mdx23c.yaml')
    conf_edit(config_path, chunk_size, overlap)


elif model == 'DENOISE-MelBand-Roformer-1 (by aufr33)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/model_mel_band_roformer_denoise.yaml'
    start_check_point = 'ckpts/denoise_mel_band_roformer_aufr33_sdr_27.9959.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/denoise_mel_band_roformer_aufr33_sdr_27.9959.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/model_mel_band_roformer_denoise.yaml')
    conf_edit(config_path, chunk_size, overlap)


elif model == 'DENOISE-MelBand-Roformer-2 (by aufr33)':
    model_type = 'mel_band_roformer'
    config_path = 'ckpts/model_mel_band_roformer_denoise.yaml'
    start_check_point = 'ckpts/denoise_mel_band_roformer_aufr33_aggr_sdr_27.9768.ckpt'
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/denoise_mel_band_roformer_aufr33_aggr_sdr_27.9768.ckpt')
    download_file('https://huggingface.co/jarredou/aufr33_MelBand_Denoise/resolve/main/model_mel_band_roformer_denoise.yaml')
    conf_edit(config_path, chunk_size, overlap)



!python inference.py \
    --model_type {model_type} \
    --config_path {config_path} \
    --start_check_point {start_check_point} \
    --input_folder '{input_folder}' \
    --store_dir '{output_folder}' \
    {('--extract_instrumental' if extract_instrumental else '')} \
    {('--flac_file' if flac_file else '')} \
    {('--use_tta' if use_tta else '')} \
    {('--pcm_type ' + pcm_type if pcm_type else '')}